<a href="https://colab.research.google.com/github/dcpatton/Image-Classification/blob/master/cifar10_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective



> Just a simple example of image classification with a CNN in TensorFlow



In [20]:
import tensorflow as tf
import random

seed = 52
tf.random.set_seed(seed)
random.seed(seed)

tf.__version__

'2.3.0'

# Get the data

In [21]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
# Normalize the data
train_images, test_images = train_images / 255.0, test_images / 255.0
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

In [22]:
print(train_images.shape)
print(test_images.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


# Model

In [23]:
from tensorflow.keras.layers import Conv2D, Flatten, MaxPool2D, Dense
from tensorflow.keras import Input, Model

inp = Input(shape=(32, 32, 3), name='inp')
x = Conv2D(64, 3, activation='relu')(inp)
x = MaxPool2D()(x)
x = Conv2D(128, 3, activation='relu')(x)
x = MaxPool2D()(x)
x = Conv2D(256, 3, activation='relu')(x)
x = MaxPool2D()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
out = Dense(10, activation='softmax', name='out')(x)

model = Model(inputs=[inp], outputs=[out])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), 
              metrics=['acc'])

model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp (InputLayer)             [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 13, 128)       73856     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 4, 4, 256)         295168    
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 2, 2, 256)        

# Training

In [24]:
filepath = 'model.h5'
mc = tf.keras.callbacks.ModelCheckpoint(filepath, save_best_only=True, 
                                        save_weights_only=True)
es = tf.keras.callbacks.EarlyStopping(patience=10, verbose=1)

history = model.fit(train_images, train_labels, epochs=200, 
                    validation_data=(test_images, test_labels), 
                    callbacks=[mc, es], verbose=1)

Epoch 1/200
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4990 - acc: 0.4476 - val_loss: 1.2097 - val_acc: 0.5668
Epoch 2/200
1563/1563 [==============================] - 6s 4ms/step - loss: 1.0489 - acc: 0.6302 - val_loss: 1.0396 - val_acc: 0.6482
Epoch 3/200
1563/1563 [==============================] - 6s 4ms/step - loss: 0.8557 - acc: 0.7015 - val_loss: 0.8867 - val_acc: 0.6917
Epoch 4/200
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7335 - acc: 0.7422 - val_loss: 0.8090 - val_acc: 0.7205
Epoch 5/200
1563/1563 [==============================] - 6s 4ms/step - loss: 0.6328 - acc: 0.7757 - val_loss: 0.7914 - val_acc: 0.7289
Epoch 6/200
1563/1563 [==============================] - 6s 4ms/step - loss: 0.5519 - acc: 0.8065 - val_loss: 0.8269 - val_acc: 0.7311
Epoch 7/200
1563/1563 [==============================] - 6s 4ms/step - loss: 0.4804 - acc: 0.8310 - val_loss: 0.8574 - val_acc: 0.7257
Epoch 8/200
1563/1563 [==============================] 

# Evaluation

In [25]:
model.load_weights(filepath)
metrics = model.evaluate(test_images, test_labels, verbose=0)
print('Loss = ' + str(metrics[0]))
print('Accuracy = ' + str(metrics[1]))

Loss = 0.7913779616355896
Accuracy = 0.7289000153541565
